Minimal code version to test why the submission fails

Version 3 : in the convolution layer, I use the option padding='same'
            => The submission fails 
            
Version 4 : in the convolution layer, no padding
            => The submission succeeds   

In [ ]:
# Write agent to file
submission_file = "submission.py"
new_text = """
def agent(observation, configuration):
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch import tensor
    import matplotlib.pyplot as plt
    import time
    from random import choice

    class CNN(nn.Module):
        def __init__(self, n_rows, n_columns):
            super(CNN, self).__init__()
            
            # with padding = same
            # => In this case, the submission to Kaggle fails
            #self.convolution1 = nn.Conv2d(in_channels = 1, out_channels = 4, kernel_size = 3,padding='same')
            #self.fc1 = nn.Linear(in_features = 168, out_features = n_columns)
            
            # withoud padding
            # => In this case, the submission to Kaggle succeeds
            self.convolution1 = nn.Conv2d(in_channels = 1, out_channels = 4, kernel_size = 3)
            self.fc1 = nn.Linear(in_features = 80, out_features = n_columns)
            
        def forward(self, x):
            x = F.relu(self.convolution1(x))
            x = x.view(x.size(0), -1)
            x = self.fc1(x)
            return x

    columns = configuration.columns
    rows = configuration.rows
    state = observation.board
    state = torch.Tensor(state).reshape((1,rows,columns)).float().unsqueeze(0)
    model = CNN(rows,columns)
    probs = F.softmax(model(state),dim=1)
    if not probs.isnan().any():
        action = probs.multinomial(num_samples=1)
        action = action.data[0,0]
    else:
        action = torch.tensor(choice([c for c in range(columns) if state[0][0][0][c] == 0]))
    if state[0][0][0][action.item()] != 0:
        action = torch.tensor(choice([c for c in range(columns) if state[0][0][0][c] == 0]))
    return action.item()
"""
with open(submission_file, "w") as f:
    f.write(new_text)

In [ ]:
# Validate agent
import sys

from kaggle_environments import agent as Kagent, utils
from kaggle_environments import make

submission = utils.read_file("/kaggle/working/submission.py")
a = Kagent.get_last_callable(submission, path=submission)

# read the file
submission = utils.read_file("/kaggle/working/submission.py")

# get the agent
agent = Kagent.get_last_callable(submission, path=submission)


In [ ]:
# play aginst itself
env = make("connectx", debug=True)
env.run([agent, agent])

print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

In [ ]:
env.render(mode = 'ipython')